In [ ]:
pip install pandas

In [ ]:
import os
import pandas as pd
import hashlib

In [ ]:
#### import, append and export PLASC exclusion data ####

folder_path = r'C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\PLASC Exclusions\\Raw Data'

csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

dfs = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(file_path)
        df = df.drop(columns=['LEA', 'Estab', 'Surname', 'Forename', 'MiddleNames', 'Gender', 'DOB', 'Sex'], errors='ignore')
        df = df.rename(columns={'StartDate': 'Exclusion_StartDate', 
                                'Reason': 'Exclusion_Reason', 
                                'ExclusionCategory': 'Exclusion_Category', 
                                'SessionsMissed': 'Exclusion_SessionsMissed'})
        dfs.append(df)
    except pd.errors.ParserError as e:
        print(f"Error reading file {file}: {e}")

merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_csv(r'C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\PLASC Exclusions\\exclusions_merged.csv', index=False)


In [ ]:
#### import, append and export PLASC pupil data ####

folder_path = r'C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\PLASC Pupil\\Raw Data'

# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# List to store already merged headers
merged_headers = []

# Columns that need to match for appending
matching_columns = ['Year', 'Estab', 'UPN', 'Gender', 'DOB', 'NCYearActual', 
                     'Ethnicity', 'SENprovision', 'FSMeligible', 
                     'PostCode', 'EALAcquisition']

# Columns to import from each CSV
columns_to_import = ['Year', 'Estab', 'UPN', 'Gender', 'DOB', 'NCYearActual', 
                     'Ethnicity', 'SENprovision', 'FSMeligible', 
                     'PostCode', 'EALAcquisition']

# Iterate over each CSV file in the folder
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if file_name.endswith('.csv'):
        # Read the CSV file
        df = pd.read_csv(file_path, dtype={'Year': str, 'Estab': str, 'UPN': str, 'Gender': str})

        # Check if 'Sex' column exists and rename it to 'Gender'
        if 'Sex' in df.columns:
            df.rename(columns={'Sex': 'Gender'}, inplace=True)

        # Ensure 'Gender' column is included
        if 'Gender' not in df.columns:
            df['Gender'] = pd.NA

        # Convert columns used for merging to string data type
        for col in matching_columns:
            if col in df.columns:
                df[col] = df[col].astype(str)

        # Check if data in specified columns match before appending
        if merged_df.empty or merged_df.astype(str).merge(df[columns_to_import], on=matching_columns, how='inner').empty:
            # Append the data to the merged DataFrame
            merged_df = pd.concat([merged_df, df[columns_to_import]], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv(r'C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\PLASC Pupil\\pupil_merged.csv', index=False)


In [ ]:
merged_df.dtypes

In [ ]:
#### combine PLASC pupil and exclusion csv files ####

# Read the CSV files
pupil_df = pd.read_csv('C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\PLASC Pupil\\pupil_merged.csv')
exclusions_df = pd.read_csv('C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\PLASC Exclusions\\exclusions_merged.csv')

# Merge the two DataFrames on the columns 'UPN' and 'Year'
merged_df = pd.merge(pupil_df, exclusions_df, on=['UPN', 'Year'], how='left')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\merged_data.csv', index=False)

In [ ]:
# Append historic attendance data with data for the current appacemic year

hist_att_df = pd.read_csv('C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\Attendance\\Attendance_2013to2023.csv')
curr_att_df = pd.read_csv('C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\Attendance\\Attendance_2023to2024.csv')

In [ ]:
hist_att_df.shape

In [ ]:
curr_att_df.shape

In [ ]:
frames = [hist_att_df, curr_att_df]
att_df = pd.concat(frames)

In [ ]:
att_df.shape

In [ ]:
att_df.head(5)

In [ ]:
merged_df.head(5)

In [ ]:
# rename 'Estab' column in merged_df
merged_df = merged_df.rename(columns={"Estab": "DES_NO"})

In [ ]:
merged_df['Year'] = merged_df['Year'].astype(str).str[:4]

In [ ]:
att_df['ACYEAR'] = att_df['ACYEAR'].astype(str).str[:4]

In [ ]:
# rename 'UP_ID' and 'ACYEAR' columns in att_df
att_df = att_df.rename(columns={"UP_ID": "UPN", "ACYEAR": "Year"})

In [ ]:
# convert datatype of 'Year' column from string to integer 
att_df['Year'] = att_df['Year'].astype(int)
merged_df['Year'] = merged_df['Year'].astype(int)

In [ ]:
# incremet the Year by 1 in the attendance dataframe
att_df['Year'] = att_df['Year'] + 1

In [ ]:
att_df.dtypes

In [ ]:
merged_df.dtypes

In [ ]:
distinct_years = att_df['Year'].unique()
distinct_years.sort()
print(distinct_years)

In [ ]:
distinct_years = merged_df['Year'].unique()
distinct_years.sort()
print(distinct_years)

In [ ]:
#### combine Attendance with merged_data.csv ####

# Merge the two DataFrames on the columns 'DES_NO', 'UPN' and 'Year'
att_merged_df = pd.merge(merged_df, att_df, on=['Year', 'DES_NO', 'UPN'], how='left')

In [ ]:
att_merged_df.head(5)

In [ ]:
att_merged_df.shape

In [ ]:
# Save the merged DataFrame to a new CSV file
att_merged_df.to_csv('C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\att_merged_data.csv', index=False)

In [ ]:
#### WIMD Postcodes rectified and spaces removed in both files outside of Python #####
#### Do THIS IN PYTHON!! ####

In [ ]:
#### combine WIMD with merged_data.csv #### Welsh Postcode

# Read the merged data CSV file
merged_df = pd.read_csv('C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\att_merged_data.csv')

In [ ]:
# Hash the UPN

# Convert column to string
#merged_df['CLIENTNUM'] = df['CLIENTNUM'].astype(str)

# Apply hashing function to the column
merged_df['UPN'] = merged_df['UPN'].apply(

    lambda x: 
        hashlib.sha256(x.encode()).hexdigest()z|
)

In [ ]:
merged_df.rename(columns={"UPN": "Hashed_UPN"})

In [ ]:
merged_df.head()

In [ ]:
# remove spaces from postcodes

att_merged_df['PostCode'] = att_merged_df['PostCode'].str.replace(' ', '')
merged_df['PostCode'] = merged_df['PostCode'].str.replace(' ', '')

In [ ]:
# Read the 'Postcode to WIMD Rank 2019.csv' file
postcode_df = pd.read_csv('C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\WIMD\\Postcode to WIMD Rank 2019.csv')

# Merge the two DataFrames on the 'Postcode' column
final_merged_df = pd.merge(merged_df, postcode_df, on='PostCode', how='left')

# Save the final merged DataFrame to a new CSV file
final_merged_df.to_csv('C:\\Users\\sgranville\\Desktop\\Uni Work\\Dissertation Research\\Data\\final_merged_data.csv', index=False)

In [ ]:
final_merged_df.head(5)

In [ ]:
final_merged_df.shape